# Homework (pandas on air dataset)
Skvorsov Kirill (M8O-108M-22)

### Tasklist:
1. Построить список уникальных типов самолетов, зарегистрированных в России
2. Какой тип самолета имеет самую позднюю дату выдчи сертификата
3. Построить запрос: Владелец Аэропорта, Аэропорт, Пассажиропоток суммарный за 2018, грузопоток суммарный за 2018
4. Перечислить аэропорты, где пассажиропоток больше медианы, а грузопоток меньше медианы
5. Перечислить авиакомпании у которых нет типов воздушных судов зарегистрированных не в России
6. Выведите список: Месяц, суммарный пассажиропоток за данный месяц, аэропорт, в котором пассажиропоток в данном месяце минимальный
7. Вывести список: тип аэропорта, средний грузопоток в месяц в аэропортах данного типа



In [73]:
"""
Распокуем архив
"""

import zipfile

def unzip_data(src_path: str, dest_path: str) -> None:
    with zipfile.ZipFile(src_path, 'r') as zip_f:
        zip_f.extractall(dest_path)


unzip_data('./air_stats.zip', './air_stats')

In [74]:
"""
Исследуем, какую смысловую нагрузку представляют из себя данные в csv файлах
"""
import os

import pandas as pd

for f_name in os.listdir('./air_stats'):
    filepath = os.path.join(os.getcwd(), 'air_stats', f_name)
    try:
        data = pd.read_csv(filepath, delimiter=',')
        data.info()
    except (pd.io.parsers.python_parser.ParserError):
        data = pd.read_csv(filepath, delimiter=';')
        data.info()
    except (pd.io.parsers.python_parser.ParserError) as e:
        print(f'Err on {filepath}: {e}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3473 entries, 0 to 3472
Data columns (total 1 columns):
 #   Column                                                                                                                                         Non-Null Count  Dtype 
---  ------                                                                                                                                         --------------  ----- 
 0   Наименование аэропорта РФ;Год периода данных;Январь;Февраль;Март;Апрель;Май;Июнь;Июль;Август;Сентябрь;Октябрь;Ноябрь;Декабрь;Январь - Декабрь  3473 non-null   object
dtypes: object(1)
memory usage: 27.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8933 entries, 0 to 8932
Data columns (total 7 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   Вид воздушного судна                           8933 non-null   object
 1   Тип 

#### 1) Построить список уникальных типов самолетов, зарегистрированных в России.

In [75]:
"""
Путем продолжительного использования поисковика и ссылаясь на википедию: 
Russian Federation: "RA-...", "RF-..." 
"""
data = pd.read_csv('./air_stats/aircraft.csv', delimiter=';')

data[data["Опознавательные знаки"].str.match('RA-.*|RF-.*')] \
    ['Тип (наименование) воздушного судна'].unique().tolist()


['Птенец-2',
 'Аватар',
 'С-2 Елочка',
 'СОКОЛ',
 'Небесный',
 'Стриж',
 'Светлый',
 'Tiger КРл-22',
 'Сура',
 'EUROSTAR',
 'КАЛИДУС-912 ПАРМА',
 'Снегирь',
 'Pioneer 330',
 'AS-105GD',
 'СП-30З О',
 'КАВАЛОН-914С',
 'АХ-8 Баталия',
 'Calidus M',
 'АХ-8 Венок русских усадеб',
 'Саргон',
 'АХ-9 REAL',
 'JANTAR STANDARD 3B-EAGLE',
 'N-77',
 'JANDAR STANDARD 3A-CONDOR',
 'BB30Z',
 'JANTAR STANDARD 2-FALCON',
 'АХ-7 "Яхрома"',
 'МД-50П',
 'АТ104',
 'JANTAR STANDARD 3D-SUPER HAWK',
 'Аэропракт-20',
 'АХ-7 "СОГАЗ"',
 'Ястреб',
 'калидус 03',
 'АХ-8 Рязань',
 'КАЛИДУС',
 'Кавалон',
 'АДД-1',
 'L-13 BLANIC-PARROT',
 'АХ-7 Праздник',
 'Ласточка',
 'АХ-8',
 'Рейнджер МПВ',
 'Ветер-2',
 'АХ-7 VIVA',
 'АХ-7 Великие Луки',
 'Икарус С42В',
 'АХ-7 ВЗЭФ',
 'С22В Икарус',
 'Авиатика-890СХ',
 'С-2УТ Синтал',
 'Х-32-912УТ Гор',
 'Поиск-06',
 'МД-50СП',
 'Калидус',
 'МД-50(с)',
 'МТО-Спорт',
 'МД-50ВГ',
 'ВА-11',
 'Peregrrine SL',
 'Патруль-09СНА',
 'EuroStar SL',
 'EuroStar SLW',
 'МД-50',
 'СП-34М',
 'Р

#### 2) Какой тип самолета имеет самую позднюю дату выдчи сертификата?

In [76]:
"""
Нужно выбрать 'Тип (наименование) воздушного судна', у которого дата самая поздняя 'дата действующего свидетельства о регистрации'. 
"""
def reverce_data(date: str) -> str:  # basic max will work fine
    return '.'.join(s for s in date.split('.')[::-1])

def _is_date_reversed(date: str) -> bool:  # to skip false reverse on retry
    return len(date.split('.')[0]) == 4

if not _is_date_reversed(data['дата действующего свидетельства о регистрации'].values[0]):
    data['дата действующего свидетельства о регистрации'] = data['дата действующего свидетельства о регистрации']\
        .apply(reverce_data)

data[data['дата действующего свидетельства о регистрации'] == \
     data['дата действующего свидетельства о регистрации'].max()] \
         ['Тип (наименование) воздушного судна'].tolist()[0]


'E 330 LXK'

#### 3) Построить запрос: Владелец Аэропорта, Аэропорт, Пассажиропоток суммарный за 2018, грузопоток суммарный за 2018.

In [77]:
"""
Для решения данного пункта нужно было понять 2 основных момента:
1) В каких csv-файлах искать
2) Нужно почистить пробелы в данных т.к. из-за них строки не совпадают. Возможно в airports.csv
можно было эвристически отбросить некоторые пояснения в столбце 'Наименование аэропорта РФ', 
но не смог придумать общего правила для этого.
"""
from typing import Optional, Union

airports = pd.read_csv('./air_stats/airports.csv', delimiter=',')
passangers_carry = pd.read_csv('./air_stats/passenger transportation.csv', delimiter=';')
cargo_carry = pd.read_csv('./air_stats/cargo transportation.csv', delimiter=';')


def remove_spaces(name: Optional[str]) -> Optional[str]:
    if not isinstance(name, str):
        return str

    return name.strip(' |\t').replace('  ', '').replace(' ', '')


airports['Наименование аэропорта РФ'] = airports['Наименование аэропорта РФ'].apply(remove_spaces)
passangers_carry['Наименование аэропорта РФ'] = passangers_carry['Наименование аэропорта РФ'].apply(remove_spaces)
cargo_carry['Наименование аэропорта РФ'] = cargo_carry['Наименование аэропорта РФ'].apply(remove_spaces)


def perform_dataset(data: pd.DataFrame) -> pd.DataFrame:
    def try_convert_dtypes_to_int(value: Union[str,float,int]) -> Union[str,int,float]:
        if value == '***':
            return 0

        try:
          return float(value.replace(',', '.').replace(' ', ''))
        except (Exception) as e:
            return value
    
    for column in data.columns:
        data[column] = data[column].apply(try_convert_dtypes_to_int)

    data = data.fillna(0)
    
    return data

passangers_carry = perform_dataset(passangers_carry)
cargo_carry = perform_dataset(cargo_carry)

passangers_carry = passangers_carry[passangers_carry['Год периода данных'] == 2018]
cargo_carry = cargo_carry[cargo_carry['Год периода данных'] == 2018]

passangers_carry['Total passangers traffic'] = passangers_carry.iloc[:, -13:-1].sum(axis=1).astype(int)
cargo_carry['Total cargo traffic'] = cargo_carry.iloc[:, -13:-1].sum(axis=1)


union_data = pd.merge(airports, passangers_carry, how='inner', on='Наименование аэропорта РФ')
union_data = pd.merge(union_data, cargo_carry, how='inner', on='Наименование аэропорта РФ')\
      .groupby(['Наименование аэропорта РФ', 'Юрлицо']).sum()[['Total passangers traffic', 'Total cargo traffic']]

union_data

,,Total passangers traffic,Total cargo traffic
Наименование аэропорта РФ,Юрлицо,,
Абакан,ОАО «Аэропорт Абакан»,150533,638.50
Айхал,"Мирнинское авиационное предприятие АК АЛРОСА"" (ОАО)""",0,0.00
Алдан,"ФКП Аэропорты Севера""""",0,0.00
Анапа(Витязево),ОАО «Аэропорт Анапа»,1288050,585.61
Апатиты(Хибины),ОАО «Аэропорт»,0,0.00
...,...,...,...
Южно-Курильск(Менделеево),КП «Аэропорты Курильских островов»,0,0.00
Южно-Сахалинск(Хомутово),"ОАО «Аэропорт Южно-Сахалинск»,",812971,7647.27
Якутск,"ОАО Аэропорт Якутск""""",707508,7055.46


#### 4) Перечислить аэропорты, где пассажиропоток больше медианы, а грузопоток меньше медианы.

In [78]:
"""
Для решения нужно найти медиану пассажиропотока и грузопотока и отфильтровать датасет из прошлого задания.
"""

passanger_carry_mean = union_data['Total passangers traffic'].mean()
cargo_carry_mean = union_data['Total cargo traffic'].mean()

airports_list = union_data[(union_data['Total passangers traffic'] > passanger_carry_mean)\
                           & (union_data['Total cargo traffic'] < cargo_carry_mean)]\
                           .index.tolist()

airports_list

[('Анапа(Витязево)', 'ОАО «Аэропорт Анапа»'),
 ('Астрахань(Нариманово)', 'ОАО «Аэропорт Астрахань»'),
 ('Воронеж(Чертовицкое)', 'ООО Управляющая компания «АВИАСЕРВИС»'),
 ('Пермь(БольшоеСавино)', 'ОАО  «Международный аэропорт Пермь»'),
 ('Петропавловск-Камчатский(Елизово)',
  'Государственное Унитарное предприятие Камчатского края «Камчатское авиационное предприятие»'),
 ('Ставрополь(Шпаковское)', 'ГУП СК «Международный аэропорт Ставрополь»'),
 ('Сыктывкар', 'ОАО  «Комиавиатранс»'),
 ('Уфа', 'ОАО «Международный аэропорт Уфа»')]

#### 5) Перечислить авиакомпании у которых нет типов воздушных судов зарегистрированных не в России

In [79]:
"""
Из прошлого задания известно, что суды принадлежащие РФ это RF-... и RA-...
Значит нужно взять список самолетов, которые подходят и проверить, есть ли такая компания, 
у которой все самолеты из списка не из РФ.
"""
import re

from typing import List


data = pd.read_csv('./air_stats/aircraft.csv', delimiter=';')

data.drop(
    data[data["Опознавательные знаки"].str.match('RA-.*|RF-.*')].index,
    inplace=True
)

print(len(data)) # 0 -> все самолеты РФ

data = pd.read_csv('./air_stats/aircraft.csv', delimiter=';')
rus_aircraft_list = data[data["Опознавательные знаки"].str.match('RA-.*|RF-.*')] \
                        ['Тип (наименование) воздушного судна'].unique().tolist()

airlines = pd.read_csv('./air_stats/airlines.csv', delimiter=',')


def check_is_foreign(aircrafts: Optional[str], rus_aircraft_list: List[str]) -> bool:
    if not isinstance(aircrafts, str):
        return False
    
    company_aircrafts_list = []  # не придумал regex для вычленения списка самолетов без индекса, поэтому хардкод
    for plane in aircrafts.split(','):
        plane = plane.split('(')[0].strip(' |,|(|)|\t')
        company_aircrafts_list.append(plane)

    has_russian_airplane = False
    for aircrft in company_aircrafts_list:
        if aircrft in rus_aircraft_list:
            has_russian_airplane = True
            break

    return not has_russian_airplane


airlines['foreign_agent'] = [True if check_is_foreign(aircrafts, rus_aircraft_list)\
                             else False\
                             for aircrafts in airlines['Самолеты']]

airlines[airlines['foreign_agent'] == True]['Название'].unique().tolist()


0


['Акционерное общество «Авиа Менеджмент Груп»',
 'Открытое акционерное общество «Авиакомпания «Аврора»',
 'ООО «Ай Флай»',
 'Общество с ограниченной ответственностью  «АТРАН»',
 'Общество с ограниченной ответственностью «Аэролимузин»',
 'ОАО «Аэрофлот - российские авиалинии»',
 'Общество с ограниченной ответственностью  «Авиакомпания «ВИМ-АВИА»',
 'ОАО «Джет Эйр Групп»',
 'Открытое акционерное общество «ДОНАВИА»',
 'Общество с ограниченной ответственностью «Авиакомпания «Икар»',
 'Общество с ограниченной ответственностью «Производственно-коммерческая фирма» «КАТЭКАВИА»',
 'Общество с ограниченной ответственностью «Авиакомпания Когалымавиа»',
 'Государственное казенное учреждение города Москвы «Московский авиационный центр»',
 'Закрытое акционерное общество «Авиакомпания «Меридиан»',
 'ЗАО «Нордавиа-региональные авиалинии»',
 'Открытое акционерное общество «Оренбургские авиалинии»',
 'Общество с ограниченной ответственностью «Авиакомпания «Победа»',
 'Открытое акционерное общество «Авиа

#### 6) Выведите список: Месяц, суммарный пассажиропоток за данный месяц, аэропорт, в котором пассажиропоток в данном месяце минимальный.

In [80]:
"""
В задании нужно работать с `passanger transportation.cvs`, но не указно нужно ли учитывать год.
Скорее всего нужно, т.к. странно считать сумму за месяц по всем годам и находить минимум с учетом
разбития данных по годам.

Тогда результатом будет список словарей [
  {
    year: [Airport_name, month, sum(passangers_for_all_airpots)]
  }
]

0. Выделим список месяцев
1. Выделим список всех лет
2. Для каждого года
  2.1 Фильтруем записи только для этого года
  2.2 Проходимся по списку месяцев и находим минимум для каждого (учитывая пропуски в виде ***)
  2.3 Находим название аэропорта, сумму за данный месяц и добавляем в результирующий список
"""
const_huge_num = 228_322_1337

data = pd.read_csv('./air_stats/passenger transportation.csv', delimiter=';')
months = data.columns[-13:-1].tolist()
years = data['Год периода данных'].unique().tolist()

def replace_to_inf(metric: Union[str,int,float]) -> Union[int,float]:
    if metric == '***':
        return const_huge_num
    
    if isinstance(metric, str):
        return float(metric.replace(' ', ''))
    
    return metric


def replace_to_zero(val: Union[float,int]) -> int:
    if val == const_huge_num:
        return 0
    
    return val


result_list = []
for year in reversed(sorted(years)):
    cur_data = data[data['Год периода данных'] == year]

    result_list_per_year = []

    cur_data = cur_data.fillna(const_huge_num)
    for month in months:
        cur_data[month] = cur_data[month].apply(replace_to_inf)
        month_min = cur_data[month].min()
        cur_year_airports = cur_data[cur_data[month] == month_min]['Наименование аэропорта РФ'].tolist()
        
        cur_data[month] = cur_data[month].apply(replace_to_zero)
        sum_for_month = cur_data[month].sum()
        if not sum_for_month:
            result_list_per_year.append((month, "No info", "No info"))
            continue

        if not len(cur_year_airports):
            result_list_per_year.append((month, sum_for_month, "No info"))
            continue

        result_list_per_year.append((month, sum_for_month, cur_year_airports[0]))

    result_list.append({year: result_list_per_year})

result_list  # Минимальные грузоперевозки там, где стоят 0, все вроде согласно условию ... 
# не сказано, что нужно отбрасывать 0

[{2018: [('Январь', 11975195.0, 'Владивосток(Кневичи)-ООО "АЭРО-ГРУЗ"'),
   ('Февраль', 10735375.0, 'Владивосток(Кневичи)-ООО "АЭРО-ГРУЗ"'),
   ('Март', 12395277.0, 'Владивосток(Кневичи)-ООО "АЭРО-ГРУЗ"'),
   ('Апрель', 13333555.0, 'Владивосток(Кневичи)-ООО "АЭРО-ГРУЗ"'),
   ('Май', 15059520.0, 'Владивосток(Кневичи)-ООО "АЭРО-ГРУЗ"'),
   ('Июнь', 18170319.0, 'Владивосток(Кневичи)-ООО "АЭРО-ГРУЗ"'),
   ('Июль', 20333167.0, 'Владивосток(Кневичи)-ООО "АЭРО-ГРУЗ"'),
   ('Август', 20693339.0, 'Владивосток(Кневичи)-ООО "АЭРО-ГРУЗ"'),
   ('Сентябрь', 18544868.0, 'Владивосток(Кневичи)-ООО "АЭРО-ГРУЗ"'),
   ('Октябрь', 'No info', 'No info'),
   ('Ноябрь', 'No info', 'No info'),
   ('Декабрь', 'No info', 'No info')]},
 {2017: [('Январь', 10768775.0, 'Майкоп'),
   ('Февраль', 9927701.0, 'Майкоп'),
   ('Март', 11156744.0, 'Майкоп'),
   ('Апрель', 12013343.0, 'Майкоп'),
   ('Май', 13779831.0, 'Майкоп'),
   ('Июнь', 16262067.0, 'Майкоп'),
   ('Июль', 18848804.0, 'Майкоп'),
   ('Август', 18791372.0, 

#### 7) Вывести список: тип аэропорта, средний грузопоток в месяц в аэропортах данного типа.

In [81]:
"""
Нужно объединить `airports.csv` и `cargo transportation.csv` по названию аэропорта 
и сгруппировать по типу аэропорта с аггрегирующей функций avg для суммы по всем месяцам
"""
airports = pd.read_csv('./air_stats/airports.csv', delimiter=',')
cargo = pd.read_csv('./air_stats/cargo transportation.csv', delimiter=';')

cargo = perform_dataset(cargo)
cargo['mean'] = cargo.iloc[:, -13:-1].sum(axis=1)

union_df = pd.merge(airports, cargo, how='inner', on='Наименование аэропорта РФ')
union_df.groupby('Тип')['mean'].mean()

Тип
Б      3825.147917
В      5307.326944
Г        98.108974
Д         0.000000
Д         0.000000
Е         0.000000
Name: mean, dtype: float64

Вывод: очень понравился датасет